# Synthesizing Data 
## By: Lela Bones
-----------------------------------
### Problem
    Nestor is an application that allows users to datify their maintenance logs. Datify is the process of taking unorginized data and putting it into a quantifiable and statistically relevant format. The problem is that because of copywright and data ownership technicalities, it is hard for us to have a large amount of "good" data to demo our app on. 

### Solution
    I plan on using Reccurent Neural Nets (RNNs) to generate realistic data from the data that we already have from companies. The reason for this is because we can use the "synthetic" data to demo our app on, and we aren't breaking any laws. I plan on using a Python library, Pytorch to implement my RNN and train it on the data. 

#### Why Pytorch???
Pytorch is a framework that builds from the Torch framework that Facebook actively uses. Pytorch is extremely fast it is very native and customizable. Pytorch is also a dynamic deep learning tool, which means that you can change and execute notes as you're learning. This makes RNNs way easier to train because you don't need to set a maximum length and then pad smaller sequences. Debugging is really easy because it is defined at runtime. It works well with Flask, which is the tool that I am using to create my visual dashboard. Pytorch also has declarative data parallelism which allows you to use multiple GPUs. 

#### Why RNNs???
    * Vanilla Neural Networks 
    * Long Short Term Memory Neural Nets are good for long-term data
    * RNNs implement loops so the learning isn't reset from iteration to iteration.

In [34]:
import pandas as pd
import torch
import numpy as np

In [41]:
csv_file = 'mine_raw.csv'
txt_file = 'train.txt'

In [44]:
def createTextFile(inputFile, outputFile):
    df = pd.read_csv(inputFile)
    data = df['OriginalShorttext'].apply(lambda x: " ".join(x.lower() for x in x.split()))

    text_list = []
    for line in data:
        text_list.append(" ".join(line))

    with open(outputFile, "a+") as output_file:
        for line in data:
            output_file.write("  " + line + "\n")
        print('File Successfully written.')

In [47]:
createTextFile(csv_file, txt_file)

File Successfully written.


In [39]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1 # Plus EOS marker


# Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]


In [52]:
dictionary = readLines(txt_file)

In [53]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)